In [1]:
import os

In [2]:
orig_images = os.listdir('/home/sadevans/space/CloseEyesClassifier/data/EyesDataset')
closed_imgs =os.listdir('/home/sadevans/space/CloseEyesClassifier/data/clustered_auto_tcne_CHECKED/close')
opened_imgs =os.listdir('/home/sadevans/space/CloseEyesClassifier/data/clustered_auto_tcne_CHECKED/open')


In [3]:
for img in orig_images:
    if img not in closed_imgs and img not in opened_imgs:
        print(img)

In [4]:
len(closed_imgs)/len(orig_images)

0.5225

In [5]:
len(opened_imgs)/len(orig_images)


0.4775

In [6]:
open_dir = '/home/sadevans/space/CloseEyesClassifier/data/clustered_auto_tcne_CHECKED/open'
close_dir = '/home/sadevans/space/CloseEyesClassifier/data/clustered_auto_tcne_CHECKED/close'

seed = 42

In [7]:
import random
from sklearn.model_selection import train_test_split


def load_image_paths_and_labels(open_dir, close_dir):
    open_images = [os.path.join(open_dir, img) for img in os.listdir(open_dir) if img.endswith(('png', 'jpg', 'jpeg'))]
    close_images = [os.path.join(close_dir, img) for img in os.listdir(close_dir) if img.endswith(('png', 'jpg', 'jpeg'))]
    images = open_images + close_images
    labels = [1] * len(open_images) + [0] * len(close_images)
    return images, labels

# Перемешать пути изображений и метки классов
def shuffle_data(images, labels):
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images[:], labels[:] = zip(*combined)
    return images, labels

# Разделить данные на обучающую, валидационную и тестовую выборки
def split_data(images, labels):
    train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.4, stratify=labels, random_state=42)
    val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)
    return train_images, val_images, test_images, train_labels, val_labels, test_labels


In [8]:
# Пример использования
# open_dir = 'path/to/open_dir'
# close_dir = 'path/to/close_dir'
import numpy as np
import torch


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# Загрузить данные
images, labels = load_image_paths_and_labels(open_dir, close_dir)

# Перемешать данные
images, labels = shuffle_data(images, labels)

# Разделить данные
train_images, val_images, test_images, train_labels, val_labels, test_labels = split_data(images, labels)

print(f"Train: {len(train_images)} images")
print(f"Validation: {len(val_images)} images")
print(f"Test: {len(test_images)} images")

Train: 2400 images
Validation: 800 images
Test: 800 images


In [9]:
train_images[2000], train_labels[2000]

('/home/sadevans/space/CloseEyesClassifier/data/clustered_auto_tcne_CHECKED/close/003373.jpg',
 0)

In [10]:
train_labels[train_labels==1]

0

In [11]:
from dataset import ImageTransform, EyeDataset
import cv2

train_dataset = EyeDataset(train_images, train_labels, transform=ImageTransform('train'))
val_dataset = EyeDataset(val_images, val_labels, transform=ImageTransform('val'))
test_dataset = EyeDataset(test_images, test_labels, transform=ImageTransform('test'))

In [12]:
from torch.utils.data import Dataset, DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
train_dataset[0]

[[[ 96  96  96]
  [108 108 108]
  [109 109 109]
  ...
  [139 139 139]
  [147 147 147]
  [145 145 145]]

 [[101 101 101]
  [103 103 103]
  [101 101 101]
  ...
  [129 129 129]
  [136 136 136]
  [137 137 137]]

 [[102 102 102]
  [ 96  96  96]
  [ 90  90  90]
  ...
  [117 117 117]
  [124 124 124]
  [129 129 129]]

 ...

 [[128 128 128]
  [128 128 128]
  [127 127 127]
  ...
  [125 125 125]
  [128 128 128]
  [129 129 129]]

 [[132 132 132]
  [132 132 132]
  [131 131 131]
  ...
  [134 134 134]
  [136 136 136]
  [134 134 134]]

 [[135 135 135]
  [135 135 135]
  [133 133 133]
  ...
  [138 138 138]
  [141 141 141]
  [139 139 139]]]
torch.Size([3, 24, 24])


{'image': tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000,  0.2391,  0.0666,  0.1136, -0.0040,  0.0273,  0.1058,
           -0.0589, -0.0668, -0.1373, -0.1609, -0.0903,  0.0117,  0.0666,
            0.0744, -1.0000, -1.0000],
          [-1.0000, -0.1609, -0.1060, -0.1217, -0.1766, -0.1766,  0.0901,
            0.4665,  0.0979, -0.0119,  0.0430, -0.0432, -0.0275, -0.0119,
           -0.0119, -0.0275, -0.1060, -0.1922, -0.2001, -0.1530, -0.0824,
           -0.0275, -0.0354, -1.0000],
          [-0.1530, -0.2079, -0.1922, -0.1609, -0.2079, -0.1844, -0.0511,
            0.1607,  0.1450, -0.0511, -0.0589, -0.0040, -0.0275, -0.0040,
           -0.0824, -0.0824, -0.1844, -0.2471, -0.2158, -0.1844, -0.1766,
           -0.1138, -0.0746, -1.0000],
          [-0.1922, -0.2942, -0.2550, -0.2158, -0.2550, -0.1530,  0.0901,
            0.2077, -0.1060, -0.1844, -0.0824, -0.0511, -0.0197,  0.0117,
           -0.0903, -0.1452, -0.2550, -0.2785, -0.2158, -0.2

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(val_dataset[0]['image'].permute(1,2,0).numpy()*255, cmap='binary')
plt.title(int(val_dataset[0]['label'].numpy()))
plt.show()